## Laboratorio 3

### A partir de la información brindada, el área de estrategia de inversiones desea desarrollar un modelo que dé señales de compra semanal que resulten en altos niveles de efectividad.

1. Evalúe el modelo de clasificación que mejores señales de compra le da a los inversionistas en el ETF cuyo precio encuentra en la base de datos.
    
    a. Analice el comportamiento de las clasificaciones de los modelos incluidos en las listas provistas.

    b. En la evaluación incluya la prueba de varios parámetros como cambiar el número de semanas de rezago en los retornos,          probar el uso de precios en vez de retornos y el cambio de alguno de los parámetros de los modelos. Ej Número de              vecinos en KNN, cálculo de número de niveles en un arbol de clasificación, número de árboles en un random forest, entre        otros. Entre todas las pruebas debe evaluar al menos 6 combinaciones de parámetros/modelos. Un ejemplo de combinaciones        sería:

        i.      KNN, con 5 vecinos, utilizando precios con 10 columnas de precios semanales
        

2. Si su objetivo es tomar decisiones de inversión para la próxima semana elija un modelo y justifique por qué. En últimas el modelo que debe elegir es el que tiene un mayor porcentaje de éxito tomando decisiones de inversión. Recuerde que si la precisión de la clasificación 1 es 50% quiere decir que la mitad de las señales de compra son falsas y la mitad son correctas. Si el recall es 50% quiere decir que el modelo va a identificar la mitad de las buenas oportunidades de compra y nos va a hacer perder la otra mitad. Recuerde jugar con la ventana de tiempo y con el umbral.

       a. Analice los resultados y justiique su elección
       b. Este ejercicio se hizo con un activo XLF, un ETF de acciones financieras. Si cambiamos el set de datos por el XLK,             de acciones tecnológicas, ¿el modelo seguiría teniendo los mismos buenos resultados? Convierta los datos con XLK en           datos de prueba x_test y y_test. 

### Entregue este laboratorio a través del Aula Virtual en formato PDF o HTML.
Asegúrese de que el archivo entregado se pueda abrir bien. En el caso de archivos HTML haga la prueba de abrirlo en algún navegador.

In [1]:
#!pip install yfinance
import yfinance as yf
precios = yf.Ticker('XLF').history(period="10y", interval='1wk')

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.svm import SVC

In [3]:
precios = precios.Close

In [4]:
precios.head(10)

Date
2013-04-08    12.386904
2013-04-15    12.125347
2013-04-22    12.453965
2013-04-29    12.641748
2013-05-06    12.910008
2013-05-13    13.379461
2013-05-20    13.231919
2013-05-27    13.305693
2013-06-03    13.392876
2013-06-10    13.111203
Name: Close, dtype: float64

In [5]:
retornos = np.log(precios).diff()[1:].dropna()
mat_retornos = retornos.values

## Ahora creamos unas listas con los modelos de clasificación que queremos incluir dentro de la función

In [ ]:
names = ["Nearest Neighbors",
    "Linear SVM",
    "Decision Tree",
    "Random Forest",
    "AdaBoost",
    "Naive Bayes"]

classifiers = [KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=50, max_features=1),
    AdaBoostClassifier(),
    GaussianNB()]

## Esta es una función para ejecutar el modelo, adapte la función para que evalúe todos los modelos incluidos en la lista anterior. Asegúrese de importar las librerías correspondientes a todos los modelos

In [121]:
def prueba_modelo(per, umbral, h):        
        
        per = 40 # ventana de tiempo - Columnas
        muestras = len(mat_retornos) - per

        variables = np.zeros((1,per))
        l = len(mat_retornos)

        for x in range(muestras):
            variables = np.concatenate((variables, mat_retornos[l-per-x: l-x].reshape((1,per))))

        variables = variables[1:]

        X = variables[:,:-1]
        Y = variables[:,-1]*100

        umbral = 2  # los retornos porcentuales en una semana determinada los vamos a clasificar como "bajo, neutral o alto"
                    # esto dependiendo del umbral que inicialmente vamos a establecerlo en 2%
            
        neutral = [not x for x in ((Y>umbral)|(Y<-umbral))]
        
        # Aquí vamos a modificar Y para clasificar los retornos -1 = bajo, 0 = neutral y 1 = alto
        # La idea es que el modelo logre clasificar efectivamente los retornos altos.
        
        Y[Y>umbral] = 1
        Y[Y<-umbral] = -1
        Y[neutral] = 0

        i_entrenamiento = int(len(variables)*0.7) #la fila hasta donde va el entrenamiento y donde comienza el conjunto de prueba
        
        # fíjese que el entrenamiento se hará utilizando datos hasta una fecha X, de ahí en adelante, probaremos el modelo
        # hacia adelante en el tiempo con lo que será el conjunto de prueba
        
        x_train = X[:i_entrenamiento, :]
        x_test =  X[i_entrenamiento:, :]
        y_train = Y[:i_entrenamiento]
        y_test =  Y[i_entrenamiento:]

        modelo = DecisionTreeClassifier(max_depth=h).fit(x_train, y_train)

        pred = modelo.predict(x_test)
    
        return pred

In [6]:
n_columnas = [5, 10, 15, 20, 25, 30]
umbral = [1,2,3,4]